In [1]:
import pandas as pd
import sqlite3
import os

In [2]:
CSV_FILE = 'titanic.csv'
DB_DIR = 'db'
DB_FILE = os.path.join(DB_DIR, 'passengers.db')

In [3]:
print("🔍 Extracting data from CSV...")
df = pd.read_csv(CSV_FILE)

🔍 Extracting data from CSV...


In [4]:
# Step 2: Transform
print("🔧 Cleaning and transforming data...")
# Drop rows with missing Age
df = df.dropna(subset=['Age'])

🔧 Cleaning and transforming data...


In [5]:
# Fill missing Embarked with mode
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

<ipython-input-5-279c65de8424>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)


In [6]:
# Normalize Fare
df['Fare'] = df['Fare'].apply(lambda x: round(x, 2))

In [7]:
# Step 3: Load
print("💾 Loading into SQLite...")
# Create directory if not exists
os.makedirs(DB_DIR, exist_ok=True)

💾 Loading into SQLite...


In [9]:
# Define the SQLite database and table name
DB_FILE = 'passengers.db'
TABLE_NAME = 'passenger_data'

In [10]:
# Connect to SQLite
conn = sqlite3.connect(DB_FILE)
df.to_sql(TABLE_NAME, conn, if_exists='replace', index=False)


714

In [11]:
print("\n📊 Summary Queries:")
cursor = conn.cursor()


📊 Summary Queries:


In [12]:
cursor.execute(f"SELECT COUNT(*) FROM {TABLE_NAME}")
print("Total passengers:", cursor.fetchone()[0])

Total passengers: 714


In [13]:
# Average fare by class
cursor.execute(f"""
    SELECT Pclass, ROUND(AVG(Fare), 2) as AvgFare
    FROM {TABLE_NAME}
    GROUP BY Pclass
""")

In [14]:
print("Average fare by class:")
for row in cursor.fetchall():
    print(f"  Class {row[0]}: €{row[1]}")

Average fare by class:
  Class 1: €87.96
  Class 2: €21.47
  Class 3: €13.23


In [15]:
conn.close()
print("\n✅ ETL pipeline completed.")


✅ ETL pipeline completed.


In [17]:
from google.colab import files
files.download(DB_FILE)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>